<a href="https://colab.research.google.com/github/Stevebankz/Hate_Speech_Detection/blob/main/Linguistically_Informed_Cross_Lingual_Meta_Learning_for_Weakly_Supervised_Hate_Speech_Detection_in_Tai_Kadai_Languages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing Hugging Face datasets and PyThaiNLP for Thai language processing
!pip install datasets
!pip install pythainlp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 103.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary libraries
from datasets import load_dataset

# Loading the Wisesight Sentiment dataset
dataset = load_dataset("pythainlp/wisesight_sentiment")

# Check the available splits (train, validation, test)
print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.58M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/286k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/327k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21628 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2404 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2671 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['texts', 'category'],
        num_rows: 21628
    })
    validation: Dataset({
        features: ['texts', 'category'],
        num_rows: 2404
    })
    test: Dataset({
        features: ['texts', 'category'],
        num_rows: 2671
    })
})


In [4]:
# random sample from the train set
sample_data = dataset['train'].shuffle(seed=42).select(range(5))  # Show 5 random samples
print(sample_data)


Dataset({
    features: ['texts', 'category'],
    num_rows: 5
})


In [5]:
# Count of occurrences of each label in the train split
label_counts = dataset['train'].features['category'].names  # Get label names (pos, neu, neg, q)

# Calculate distribution of labels
label_distribution = dataset['train'].to_pandas()['category'].value_counts()
print(label_distribution)


category
1    11795
2     5491
0     3866
3      476
Name: count, dtype: int64


In [6]:
# Analyze the length of the messages (number of characters and words)
import pandas as pd

# Convert the dataset to a pandas DataFrame
df = dataset['train'].to_pandas()

# Calculate message lengths
df['char_length'] = df['texts'].apply(len)  # Length in characters
df['word_length'] = df['texts'].apply(lambda x: len(x.split()))  # Length in words

# Display basic statistics about message length
print(df[['char_length', 'word_length']].describe())


        char_length   word_length
count  21628.000000  21628.000000
mean      89.818291      6.340300
std      149.255596     12.186247
min        1.000000      1.000000
25%       19.000000      1.000000
50%       39.000000      3.000000
75%       98.000000      6.000000
max     1997.000000    292.000000


In [7]:
# Install PyThaiNLP tokenizer
from pythainlp.tokenize import word_tokenize

# Tokenizing a Thai sentence
sample_text = df['texts'].iloc[0]  # Taking the first text from the dataset
tokenized_text = word_tokenize(sample_text, engine='newmm')
print(tokenized_text)  # View the tokenized text


['ไป', 'จอง', 'มา', 'แล้', 'วนา', 'จา', ' ', 'Mitsubishi', ' ', 'Attrage', ' ', 'ได้', 'หลัง', 'สงกรานต์', 'เลย', ' ', 'รอ', 'ขับ', 'อยู่', 'นา', 'จา', ' ', 'กระทัดรัด', ' ', 'เหมาะกับ', 'สาว', 'ๆ', 'ขับรถ', 'คนเดียว', 'แบบ', 'เรา', ' ', 'ราคา', 'สบาย', 'กระเป๋า', ' ', 'ประหยัด', 'น้ำมัน', ' ', 'วิ่ง', 'ไกล', 'แค่', 'ไหน', 'หายห่วง', 'ค่ะ']


In [8]:
# Function to preprocess a single text
def preprocess_text(text):
    # Tokenize the text using PyThaiNLP (newmm is the default tokenizer)
    tokens = word_tokenize(text, engine='newmm')

    # Lowercase all tokens
    tokens = [token.lower() for token in tokens]

    # Join tokens back into a string
    return ' '.join(tokens)

# Apply preprocessing function to the dataset
df['processed_texts'] = df['texts'].apply(preprocess_text)

# View the first few rows of the preprocessed dataset
print(df[['texts', 'processed_texts']].head())


                                               texts  \
0  ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกร...   
1  เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึ...   
2                           บัตรสมาชิกลดได้อีกไหมคับ   
3                                สนใจ new mazda2ครับ   
4                                                 😍😍   

                                     processed_texts  
0  ไป จอง มา แล้ วนา จา   mitsubishi   attrage   ...  
1  เปิด ศักราช ใหม่ !   นายกฯ   แถลงข่าว ก่อน การ...  
2                      บัตรสมาชิก ลด ได้ อีก ไหม คับ  
3                          สนใจ   new   mazda 2 ครับ  
4                                                 😍😍  


In [9]:
# Save the preprocessed data to a CSV file for further use
df[['processed_texts', 'category']].to_csv('preprocessed_wisesight_sentiment.csv', index=False)


In [10]:
import pandas as pd

# Load the toxic words from the text file
with open('/content/drive/MyDrive/toxic_keywords_1.txt', 'r') as f:
    toxic_words = [line.strip() for line in f]

# Convert the list to a pandas Series (optional, for consistency)
toxic_words_df = pd.DataFrame(toxic_words, columns=['word'])

# Preview the toxic word list
print(toxic_words[:10])  # Display the first 10 toxic words

['ควาย', 'ทุเรศ', 'หมา', 'ตอแหล', 'โง่', 'ต่ำ', 'บ้า', 'เห็บ', 'เลว', 'กะหรี่']


In [11]:
# Create a function to check if a text contains any toxic words
def contains_toxic_words(text, toxic_words):
    # Tokenize the text using the same tokenizer (PyThaiNLP) used in preprocessing
    tokens = word_tokenize(text, engine='newmm')
    # Check if any token is in the toxic word list
    return any(token in toxic_words for token in tokens)

# Apply weak labeling based on sentiment and toxic words
def weak_labeling(row, toxic_words):
    # If the sentiment is negative, we flag it as potential hate speech (weakly labeled as 1)
    if row['category'] == 'neg':
        return 1
    # If the message contains toxic words, we flag it as potential hate speech (weakly labeled as 1)
    elif contains_toxic_words(row['processed_texts'], toxic_words):
        return 1
    # Otherwise, we consider it non-hateful (weakly labeled as 0)
    else:
        return 0

# Apply the weak labeling function to the dataset
df['weak_label'] = df.apply(weak_labeling, axis=1, toxic_words=toxic_words)

# Preview the newly labeled dataset
print(df[['processed_texts', 'category', 'weak_label']].head())


                                     processed_texts  category  weak_label
0  ไป จอง มา แล้ วนา จา   mitsubishi   attrage   ...         1           0
1  เปิด ศักราช ใหม่ !   นายกฯ   แถลงข่าว ก่อน การ...         1           0
2                      บัตรสมาชิก ลด ได้ อีก ไหม คับ         1           0
3                          สนใจ   new   mazda 2 ครับ         1           0
4                                                 😍😍         0           0


In [12]:
# Check the distribution of weak labels (0 = non-hateful, 1 = potentially hateful)
weak_label_distribution = df['weak_label'].value_counts()
print(weak_label_distribution)


weak_label
0    20783
1      845
Name: count, dtype: int64


In [13]:
# Save the weakly labeled dataset to a CSV file for further use
df[['processed_texts', 'weak_label']].to_csv('weakly_labeled_wisesight_sentiment.csv', index=False)


In [14]:
!pip install imbalanced-learn
import pandas as pd
from imblearn.over_sampling import RandomOverSampler

# Load the weakly labeled data (assuming it's saved as a CSV)
df = pd.read_csv('weakly_labeled_wisesight_sentiment.csv')

# Separate features and labels
texts = df['processed_texts']
labels = df['weak_label']

# Reshape data for oversampling
texts_reshaped = texts.values.reshape(-1, 1)  # Needs to be 2D for oversampling

# Initialize the RandomOverSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=42)

# Perform oversampling
texts_resampled, labels_resampled = ros.fit_resample(texts_reshaped, labels)

# Convert resampled data back to a DataFrame
balanced_df = pd.DataFrame({
    'texts': texts_resampled.flatten(),  # Flatten to 1D
    'weak_label': labels_resampled
})

# Save or use the balanced dataset directly for further steps
balanced_df.to_csv('balanced_weakly_labeled_data.csv', index=False)
print(balanced_df['weak_label'].value_counts())


weak_label
0    20783
1    20783
Name: count, dtype: int64


In [15]:
# Install Hugging Face transformers library
!pip install transformers
!pip install datasets
!pip install torch  # PyTorch for model training

In [16]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Load the tokenizer for a multilingual model (mBERT in this case)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Function to tokenize text
def tokenize_data(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

# Load the balanced dataset (from the CSV saved after oversampling)
balanced_df = pd.read_csv('balanced_weakly_labeled_data.csv')

# Split the dataset into train, validation, and test sets
train_val_df, test_df = train_test_split(balanced_df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42)

# Tokenize each dataset split and capture both 'input_ids' and 'attention_mask'
train_encodings = tokenize_data(train_df['texts'].tolist())
val_encodings = tokenize_data(val_df['texts'].tolist())
test_encodings = tokenize_data(test_df['texts'].tolist())

# Separate 'input_ids' and 'attention_mask' for each split
train_texts, train_masks = train_encodings['input_ids'], train_encodings['attention_mask']
val_texts, val_masks = val_encodings['input_ids'], val_encodings['attention_mask']
test_texts, test_masks = test_encodings['input_ids'], test_encodings['attention_mask']

# Convert labels to PyTorch tensors
train_labels = torch.tensor(train_df['weak_label'].tolist())
val_labels = torch.tensor(val_df['weak_label'].tolist())
test_labels = torch.tensor(test_df['weak_label'].tolist())


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
# Custom Dataset class with attention mask support
class TextDataset(Dataset):
    def __init__(self, texts, masks, labels):
        self.texts = texts
        self.masks = masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.texts[idx],
            'attention_mask': self.masks[idx],
            'labels': self.labels[idx]
        }

# Create train, validation, and test datasets with attention masks
train_dataset = TextDataset(train_texts, train_masks, train_labels)
val_dataset = TextDataset(val_texts, val_masks, val_labels)
test_dataset = TextDataset(test_texts, test_masks, test_labels)

# Create DataLoader objects for batching
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # No shuffle for test


In [18]:
from transformers import AutoModelForSequenceClassification, AdamW

# Load pre-trained mBERT model for classification (binary classification: hate vs. non-hate)
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=2  # Binary classification (hate vs. non-hate)
)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [19]:
!pip install torchmetrics
import torch.nn as nn
from torch.optim import AdamW

# Define label smoothing cross-entropy loss
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing

    def forward(self, outputs, targets):
        # Number of classes (e.g., 2 for binary classification)
        n_classes = outputs.size(-1)

        # Apply log_softmax to outputs
        log_probs = torch.nn.functional.log_softmax(outputs, dim=-1)

        # Create smooth labels
        targets = torch.zeros_like(log_probs).scatter(1, targets.unsqueeze(1), 1)  # One-hot encode targets
        targets = (1 - self.smoothing) * targets + self.smoothing / n_classes  # Apply label smoothing

        # Compute the smoothed cross-entropy loss
        loss = (-targets * log_probs).mean()
        return loss

# Define optimizer with PyTorch's AdamW to avoid the deprecation warning
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define a simple training loop with label smoothing
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=3):
    model.train()

    for epoch in range(num_epochs):
        total_loss, total_val_loss = 0, 0
        correct, total = 0, 0

        # Training phase
        for batch in train_loader:
            optimizer.zero_grad()

            # Move batch data to GPU if available
            input_ids = batch['input_ids'].squeeze(1).to(device)
            attention_mask = batch['attention_mask'].squeeze(1).to(device)  # Added attention mask
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits  # Include attention_mask
            loss = criterion(outputs, labels)  # Use custom criterion with label smoothing

            # Backward pass
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Validation phase
        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].squeeze(1).to(device)
                attention_mask = batch['attention_mask'].squeeze(1).to(device)  # Added attention mask
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits  # Include attention_mask
                val_loss = criterion(outputs, labels)  # Use custom criterion with label smoothing

                total_val_loss += val_loss.item()

                # Calculate accuracy
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        avg_train_loss = total_loss / len(train_loader)
        avg_val_loss = total_val_loss / len(val_loader)
        val_accuracy = correct / total

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}, Val Accuracy: {val_accuracy}")

# Train the model with label smoothing
train_model(model, train_loader, val_loader, criterion=LabelSmoothingCrossEntropy(smoothing=0.1), optimizer=optimizer, num_epochs=3)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 52.6 MB/s eta 0:00:00
Epoch 1/3, Train Loss: 0.1474396990246788, Val Loss: 0.11015640469825166, Val Accuracy: 0.9887730553327987
Epoch 2/3, Train Loss: 0.10660059831713942, Val Loss: 0.11256408228132969, Val Accuracy: 0.9859663191659984
Epoch 3/3, Train Loss: 0.10421657228154463, Val Loss: 0.10319312343485335, Val Accuracy: 0.9965249933172948


In [20]:
from sklearn.metrics import classification_report
import torch
import numpy as np

# Initialize variables for tracking test loss and correct predictions
test_loss = 0
correct, total = 0, 0

# Store predictions and true labels
all_preds = []
y_true = []

# Set model to evaluation mode
model.eval()
with torch.no_grad():
    for batch in test_loader:  # Use val_loader if you don't have a separate test set
        input_ids = batch['input_ids'].squeeze(1).to(device)
        labels = batch['labels'].to(device)

        # Forward pass (no labels argument here)
        outputs = model(input_ids=input_ids)
        logits = outputs.logits

        # Calculate loss
        criterion = LabelSmoothingCrossEntropy(smoothing=0.1)  # Use the same criterion as training
        loss = criterion(logits, labels)
        test_loss += loss.item()

        # Get predictions
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())  # Store predictions
        y_true.extend(labels.cpu().numpy())    # Store true labels

        # Calculate accuracy
        correct += (preds == labels).sum().item()
        total += labels.size(0)

# Average test loss
avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct / total

print(f"Test Loss: {avg_test_loss}, Test Accuracy: {test_accuracy}")

# Generate classification report
y_true = np.array(y_true)  # Convert true labels to numpy array
y_pred = np.array(all_preds)  # Convert predictions to numpy array
print(classification_report(y_true, y_pred, target_names=['Non-hate', 'Hate']))


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Test Loss: 0.12603263915158236, Test Accuracy: 0.9735386095742121
              precision    recall  f1-score   support

    Non-hate       0.96      0.99      0.97      2153
        Hate       0.99      0.95      0.97      2004

    accuracy                           0.97      4157
   macro avg       0.98      0.97      0.97      4157
weighted avg       0.97      0.97      0.97      4157



In [21]:
# Example of identifying misclassified samples
misclassified = [
    (input_ids, true, pred)
    for input_ids, true, pred in zip(test_texts, y_true, y_pred)  # or val_texts if you're analyzing validation
    if true != pred
]

# Display the first 10 misclassified examples in readable format
for input_ids, true, pred in misclassified[:10]:  # First 10 misclassified examples
    # Detokenize to retrieve original text
    text = tokenizer.decode(input_ids, skip_special_tokens=True)
    print(f"Text: {text}\nTrue Label: {true}, Predicted: {pred}\n")


Text: ฆ่า หั่น ศพ เปรี้ยว หนี ตั้ง หลาย วัน นั่ง คุย กับ ตำรวจ สบายใจ อันนี้ บุหรี่ ไฟฟ้า จับ ลาก ส่ะ
True Label: 1, Predicted: 0

Text: ฆ่า หั่น ศพ เปรี้ยว หนี ตั้ง หลาย วัน นั่ง คุย กับ ตำรวจ สบายใจ อันนี้ บุหรี่ ไฟฟ้า จับ ลาก ส่ะ
True Label: 1, Predicted: 0

Text: แกง กะหรี่ ไข่ ข้น แซบ เวอร์ กด like กัน ไป ยาว ๆ # deliciouslyyours # showdc
True Label: 1, Predicted: 0

Text: ไฮไลท์ ที่ พลาด ไม่ ได้ ของ สวนสัตว์ ขอนแก่น คือ การ เดิน ชม สัตว อัฟริกา บน ทางเดิน ลอยฟ้า ทั้ง มุมมอง สัตว์ จาก ที่สูง และ จุดชมวิว ที่ งดงาม สุด ประทับใจ
True Label: 1, Predicted: 0

Text: ทีม นักวิทยาศาสตร์ ค้นพบ เสือดำ หา ยาก ใน เคนยา นับ เป็นครั้งแรก ที่ พบ เสือ ชนิด นี้ ใน แอฟริกา ใน รอบ เกือบ 100 ปี # ข่าว โม โน 29 # เสือดำ # แอฟริกา # สัตว์ หา ยาก # mono29 # mono29news
True Label: 1, Predicted: 0

Text: ไม่ เจียม ไง เสือก แดก ลีโอ
True Label: 1, Predicted: 0

Text: ฆ่า คนตาย โทษเบา กว่า ตั้ง เยอะ
True Label: 1, Predicted: 0

Text: แสงโสม นี่ พัก เน้อ ตื่น ยาก ฉิบหาย ขำ หัว อี ก.
True Label: 1, Predicted

In [22]:
# Specify the directory in Google Drive where you want to save the model
save_directory = '/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW'

# Save the fine-tuned model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")


Model and tokenizer saved to /content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW


In [ ]:
# Specify destination directory in Google Drive
import shutil
save_directory = '/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW'

# Ensure the destination directory exists
!mkdir -p "{save_directory}"

# Paths to local files
files_to_save = {
    "balanced_weakly_labeled_data.csv": "/content/balanced_weakly_labeled_data.csv",
    "preprocessed_wisesight_sentiment.csv": "/content/preprocessed_wisesight_sentiment.csv",
    "toxic_keywords_1.txt": "/content/toxic_keywords_1.txt",
    "weakly_labeled_wisesight_sentiment.csv": "/content/weakly_labeled_wisesight_sentiment.csv"
}


In [23]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load your pre-trained BERT model and tokenizer
model_path = '/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [24]:
# Load your Thai dataset (replace with your actual file path)
data_path = '/content/drive/MyDrive/balanced_weakly_labeled_data.csv'
df = pd.read_csv(data_path)

# Clean text function to remove non-Thai characters (optional, depends on dataset)
import re
def clean_text(text):
    return re.sub(r"[^ก-๙\s]", "", text)  # Retain only Thai characters and spaces

# Apply text cleaning
df['cleaned_texts'] = df['texts'].apply(clean_text)

# Tokenize the cleaned text using the loaded tokenizer
def tokenize_texts(texts):
    return tokenizer(
        texts,
        padding=True,          # Add padding to ensure equal length sequences
        truncation=True,       # Truncate long sequences to max length
        max_length=128,        # Adjust max length based on model needs
        return_tensors="pt"    # Return as PyTorch tensors
    )

# Tokenize the dataset
tokenized_data = tokenize_texts(df['cleaned_texts'].tolist())
print(f"Tokenized input IDs shape: {tokenized_data['input_ids'].shape}")


Tokenized input IDs shape: torch.Size([41566, 128])


In [25]:
from torch.utils.data import Dataset, DataLoader

# Define a custom Dataset class for the tokenized data
class ThaiCharacterDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Convert labels to tensors
labels = torch.tensor(df['weak_label'].tolist())

# Create Dataset and DataLoader
thai_dataset = ThaiCharacterDataset(tokenized_data['input_ids'], tokenized_data['attention_mask'], labels)
thai_dataloader = DataLoader(thai_dataset, batch_size=16, shuffle=True)


In [26]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss()

# Training loop
num_epochs = 3
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in thai_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(thai_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")


Epoch 1/3, Loss: 0.026647572094142428
Epoch 2/3, Loss: 0.01543018503167546
Epoch 3/3, Loss: 0.013409500719137747


In [27]:


# Define the directory in Google Drive where you want to save the model
save_directory = '/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW'

# Save the model
model.base_model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model and tokenizer saved to /content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW


In [28]:
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoConfig

# Load your fine-tuned BERT model
model_path = '/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW'
base_model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)

# Define a new model with an additional phoneme prediction head
class MultiTaskBERT(nn.Module):
    def __init__(self, base_model, phoneme_vocab_size):
        super(MultiTaskBERT, self).__init__()
        self.base_model = base_model  # Main model for hate speech classification
        self.phoneme_head = nn.Linear(base_model.config.hidden_size, phoneme_vocab_size)  # Phoneme prediction head

    def forward(self, input_ids, attention_mask):
        # Get outputs from the BERT model with hidden states
        outputs = self.base_model.bert(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]  # Last hidden layer for phoneme prediction

        # Phoneme prediction output (character-level)
        phoneme_logits = self.phoneme_head(last_hidden_state)  # Shape: (batch_size, seq_len, phoneme_vocab_size)

        # Hate speech classification output
        cls_logits = self.base_model.classifier(outputs.pooler_output)  # Shape: (batch_size, num_labels)

        return phoneme_logits, cls_logits

# Set phoneme vocabulary size (customize based on your data)
phoneme_vocab_size = 100  # Adjust based on actual phoneme classes
model = MultiTaskBERT(base_model, phoneme_vocab_size).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
import pandas as pd
from pythainlp.tokenize import syllable_tokenize
import torch

# Load your Thai dataset
data_path = '/content/drive/MyDrive/balanced_weakly_labeled_data.csv'
df = pd.read_csv(data_path)

# Step 1: Generate syllable tokens as phoneme-like approximations
def generate_syllable_tokens(text):
    syllables = syllable_tokenize(text, engine='dict')  # Use syllabic segmentation
    return syllables

df['syllable_tokens'] = df['texts'].apply(generate_syllable_tokens)

# Step 2: Create a syllable vocabulary and map syllables to unique IDs
syllable_vocab = {}
current_id = 0

for syllables in df['syllable_tokens']:
    for syllable in syllables:
        if syllable not in syllable_vocab:
            syllable_vocab[syllable] = current_id
            current_id += 1

# Step 3: Convert syllable sequences to integer sequences
def convert_to_ids(sequence):
    return [syllable_vocab[syllable] for syllable in sequence]

df['syllable_ids'] = df['syllable_tokens'].apply(convert_to_ids)

# Step 4: Pad syllable sequences to a fixed length (e.g., 128)
max_len = 128
def pad_sequence(seq):
    if len(seq) < max_len:
        return seq + [0] * (max_len - len(seq))  # Pad with 0s for shorter sequences
    else:
        return seq[:max_len]  # Truncate longer sequences

df['syllable_ids_padded'] = df['syllable_ids'].apply(pad_sequence)

# Convert to tensor for training
phoneme_labels = torch.tensor(df['syllable_ids_padded'].tolist())
print("Phoneme-like labels shape:", phoneme_labels.shape)


Phoneme-like labels shape: torch.Size([41566, 128])


In [30]:
phoneme_labels = torch.randint(0, phoneme_vocab_size, (len(df), 128))


In [31]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

# Set up optimizer and loss functions
optimizer = AdamW(model.parameters(), lr=2e-5)
phoneme_loss_fn = CrossEntropyLoss()
classification_loss_fn = CrossEntropyLoss()

# Training loop with dual losses
num_epochs = 3
model.train()

for epoch in range(num_epochs):
    total_phoneme_loss, total_classification_loss = 0, 0
    for batch_idx, batch in enumerate(thai_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        classification_labels = batch['labels'].to(device)

        # Retrieve the corresponding phoneme labels for the current batch
        phoneme_label_batch = phoneme_labels[batch_idx * len(input_ids):(batch_idx + 1) * len(input_ids)].to(device)

        optimizer.zero_grad()

        # Forward pass
        phoneme_logits, cls_logits = model(input_ids=input_ids, attention_mask=attention_mask)

        # Reshape phoneme_logits and phoneme_label_batch to match dimensions
        phoneme_logits = phoneme_logits.view(-1, phoneme_vocab_size)  # Shape: (batch_size * seq_len, vocab_size)
        phoneme_label_batch = phoneme_label_batch.view(-1)  # Shape: (batch_size * seq_len)

        # Calculate phoneme loss
        phoneme_loss = phoneme_loss_fn(phoneme_logits, phoneme_label_batch)

        # Calculate classification loss
        classification_loss = classification_loss_fn(cls_logits, classification_labels)

        # Combine losses and optimize
        total_loss = phoneme_loss + classification_loss
        total_loss.backward()
        optimizer.step()

        total_phoneme_loss += phoneme_loss.item()
        total_classification_loss += classification_loss.item()

    avg_phoneme_loss = total_phoneme_loss / len(thai_dataloader)
    avg_classification_loss = total_classification_loss / len(thai_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Phoneme Loss: {avg_phoneme_loss}, Classification Loss: {avg_classification_loss}")


Epoch 1/3, Phoneme Loss: 4.6098831188137295, Classification Loss: 0.01307484107452516
Epoch 2/3, Phoneme Loss: 4.607374034908021, Classification Loss: 0.008522843429922931
Epoch 3/3, Phoneme Loss: 4.606879003604069, Classification Loss: 0.01000913630996141


In [32]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

# Set up optimizer and loss functions
optimizer = AdamW(model.parameters(), lr=2e-5)
phoneme_loss_fn = CrossEntropyLoss()
classification_loss_fn = CrossEntropyLoss()

# Training loop with dual losses
num_epochs = 3
model.train()

for epoch in range(num_epochs):
    total_phoneme_loss, total_classification_loss = 0, 0
    for batch in thai_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        classification_labels = batch['labels'].to(device)

        # Retrieve the corresponding phoneme labels for the current batch
        phoneme_label_batch = phoneme_labels[batch['labels']]  # Ensure this matches batch size
        phoneme_label_batch = phoneme_label_batch.to(device)  # Move phoneme labels to device

        optimizer.zero_grad()

        # Forward pass
        phoneme_logits, cls_logits = model(input_ids=input_ids, attention_mask=attention_mask)

        # Reshape phoneme_logits and phoneme_label_batch to match dimensions
        phoneme_logits = phoneme_logits.view(-1, phoneme_vocab_size)  # Shape: (batch_size * seq_len, vocab_size)
        phoneme_label_batch = phoneme_label_batch.view(-1)  # Shape: (batch_size * seq_len)

        # Calculate phoneme loss
        phoneme_loss = phoneme_loss_fn(phoneme_logits, phoneme_label_batch)

        # Calculate classification loss
        classification_loss = classification_loss_fn(cls_logits, classification_labels)

        # Combine losses and optimize
        total_loss = phoneme_loss + classification_loss
        total_loss.backward()
        optimizer.step()

        total_phoneme_loss += phoneme_loss.item()
        total_classification_loss += classification_loss.item()

    avg_phoneme_loss = total_phoneme_loss / len(thai_dataloader)
    avg_classification_loss = total_classification_loss / len(thai_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Phoneme Loss: {avg_phoneme_loss}, Classification Loss: {avg_classification_loss}")


Epoch 1/3, Phoneme Loss: 3.639465255632687, Classification Loss: 0.024973187675885036
Epoch 2/3, Phoneme Loss: 0.1615475686381731, Classification Loss: 0.01320341678564209
Epoch 3/3, Phoneme Loss: 0.032623962771285146, Classification Loss: 0.008391529086575604


In [33]:

import os
import torch


# Define the save path in Google Drive
save_directory = '/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW'

# Ensure the directory exists
os.makedirs(save_directory, exist_ok=True)

# Save the model's state dictionary
model_path = os.path.join(save_directory, "pytorch_model.bin")
torch.save(model.state_dict(), model_path)

# Save the model configuration if applicable (e.g., if you used a Hugging Face configuration object)
if hasattr(model, "config"):
    model.config.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model state dictionary and tokenizer saved to {save_directory}")


Model state dictionary and tokenizer saved to /content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW


In [34]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define the save path in Google Drive
model_path = '/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW'
state_dict_path = f"{model_path}/pytorch_model.bin"

# Step 1: Initialize the model architecture
config_path = "bert-base-multilingual-cased"  # Original architecture
model = AutoModelForSequenceClassification.from_pretrained(config_path, num_labels=2)  # Adjust labels as needed

# Step 2: Load the saved state dictionary with strict=False
state_dict = torch.load(state_dict_path)
model.load_state_dict(state_dict, strict=False)  # Load only matching parameters

# Step 3: Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(config_path)

# Move model to the available device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model and tokenizer loaded successfully with partial loading!")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-34-ab372d224baf>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globa

Model and tokenizer loaded successfully with partial loading!


In [35]:
# Define phoneme prediction layer
class MultiTaskBERT(nn.Module):
    def __init__(self, base_model, phoneme_vocab_size):
        super(MultiTaskBERT, self).__init__()
        self.base_model = base_model
        self.phoneme_head = nn.Linear(self.base_model.config.hidden_size, phoneme_vocab_size)  # Phoneme prediction head

    def forward(self, input_ids, attention_mask, labels=None, phoneme_labels=None):
        outputs = self.base_model.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        phoneme_logits = self.phoneme_head(last_hidden_state)  # Shape: (batch_size, seq_len, phoneme_vocab_size)

        return phoneme_logits

# Instantiate the MultiTask model
phoneme_vocab_size = 30  # Example size (adjust based on phoneme data)
multi_task_model = MultiTaskBERT(model, phoneme_vocab_size).to(device)

In [36]:
import torch.nn as nn
from transformers import AutoModel

# Assuming your fine-tuned model is saved in 'model_path'
model_path = '/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW'
base_model = AutoModel.from_pretrained(model_path, output_hidden_states=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)

# Define a new model with an additional POS tagging head using the pre-fine-tuned BERT model
class MultiTaskBERTWithPOS(nn.Module):
    def __init__(self, base_model, pos_vocab_size):
        super(MultiTaskBERTWithPOS, self).__init__()
        self.base_model = base_model
        self.pos_head = nn.Linear(self.base_model.config.hidden_size, pos_vocab_size)  # POS tagging head

    def forward(self, input_ids, attention_mask, labels=None, pos_labels=None, output_hidden_states=False):
        # Call the base model directly
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=output_hidden_states)

        # Access last hidden state directly from dictionary output
        last_hidden_state = outputs['last_hidden_state']
        hidden_states = outputs.get('hidden_states', None)  # Use .get() to avoid KeyError if hidden_states are not present

        # POS tagging logits from the last hidden state
        pos_logits = self.pos_head(last_hidden_state)  # Shape: (batch_size, seq_len, pos_vocab_size)

        # Return dictionary with all relevant outputs
        return {
            "last_hidden_state": last_hidden_state,
            "pos_logits": pos_logits,
            "hidden_states": hidden_states
        }

# Initialize with your model and specified POS vocabulary size
pos_vocab_size = 50  # Adjust based on actual POS vocabulary size
multi_task_model = MultiTaskBERTWithPOS(base_model=base_model, pos_vocab_size=pos_vocab_size).to(device)

In [37]:
import torch
from torch.utils.data import Dataset, DataLoader
from pythainlp.tag import pos_tag
from pythainlp import word_tokenize
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW")

# Dummy POS vocabulary (expand with actual vocabulary from your dataset)
pos_vocab = {"NOUN": 0, "VERB": 1, "ADJ": 2, "ADV": 3, "PRON": 4, "DET": 5, "CONJ": 6, "NUM": 7, "PUNCT": 8}
pos_vocab_size = len(pos_vocab)

# Function to tokenize and get POS tags
def tokenize_and_pos_tag(text):
    tokens = word_tokenize(text, engine="newmm")  # Tokenize
    pos_tags = [tag for word, tag in pos_tag(tokens, corpus="orchid")]  # POS tagging using PyThaiNLP
    return tokens, pos_tags

# Function to convert POS tags to indices
def pos_to_indices(pos_tags):
    return [pos_vocab.get(tag, pos_vocab["NOUN"]) for tag in pos_tags]  # Default to "NOUN" if tag is out of vocabulary

# Custom Dataset for Multi-Task Learning
class ThaiMultiTaskDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize and get POS tags
        tokens, pos_tags = tokenize_and_pos_tag(self.texts[idx])
        pos_indices = pos_to_indices(pos_tags)

        # Encode tokens using the tokenizer
        encoding = tokenizer(tokens, is_split_into_words=True, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

        # Directly retrieve tensors
        input_ids = encoding['input_ids'].squeeze(0)  # Remove batch dimension
        attention_mask = encoding['attention_mask'].squeeze(0)

        # Pad or truncate POS tags to match the max length (128)
        pos_labels = torch.tensor(pos_indices[:128] + [0] * (128 - len(pos_indices)))

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'pos_labels': pos_labels
        }

# Prepare DataLoader
texts = df['texts'].tolist()  # Assuming your texts are in a DataFrame column named 'texts'
thai_dataset = ThaiMultiTaskDataset(texts)
thai_dataloader = DataLoader(thai_dataset, batch_size=16, shuffle=True)

print("DataLoader created with POS labels for Thai dataset.")


DataLoader created with POS labels for Thai dataset.


In [38]:
from torch.optim import AdamW

# Initialize Multi-Task Model with POS tagging head
multi_task_model = MultiTaskBERTWithPOS(base_model=model, pos_vocab_size=pos_vocab_size).to(device)

# Define optimizer and loss functions
optimizer = AdamW(multi_task_model.parameters(), lr=2e-5)
classification_loss_fn = nn.CrossEntropyLoss()
pos_loss_fn = nn.CrossEntropyLoss()

# Training loop with dual objectives
num_epochs = 3
multi_task_model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in thai_dataloader:
        optimizer.zero_grad()

        # Use tensors directly as they’re already in the right form
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pos_labels = batch['pos_labels'].to(device)

        # Forward pass
        outputs = multi_task_model(input_ids=input_ids, attention_mask=attention_mask)

        # Access the outputs using the correct keys
        last_hidden_state = outputs["last_hidden_state"]
        pos_logits = outputs["pos_logits"]

        # Calculate POS tagging loss
        pos_loss = pos_loss_fn(pos_logits.view(-1, pos_vocab_size), pos_labels.view(-1))

        # Total loss for multi-task
        loss = pos_loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(thai_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")

KeyError: 'last_hidden_state'

In [39]:
import torch.nn as nn
from transformers import AutoModel

# Assuming your fine-tuned model is saved in 'model_path'
model_path = '/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW'
base_model = AutoModel.from_pretrained(model_path, output_hidden_states=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)

# Define a new model with an additional POS tagging head using the pre-fine-tuned BERT model
class MultiTaskBERTWithPOS(nn.Module):
    def __init__(self, base_model, pos_vocab_size):
        super(MultiTaskBERTWithPOS, self).__init__()
        self.base_model = base_model
        self.pos_head = nn.Linear(self.base_model.config.hidden_size, pos_vocab_size)  # POS tagging head

    def forward(self, input_ids, attention_mask, labels=None, pos_labels=None, output_hidden_states=False):
        # Call the base model directly
        # Changed model to base_model
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=output_hidden_states)

        # Access last hidden state directly from dictionary output
        last_hidden_state = outputs['last_hidden_state']
        hidden_states = outputs.get('hidden_states', None)  # Use .get() to avoid KeyError if hidden_states are not present

        # POS tagging logits from the last hidden state
        pos_logits = self.pos_head(last_hidden_state)  # Shape: (batch_size, seq_len, pos_vocab_size)

        # Return dictionary with all relevant outputs
        return {
            "last_hidden_state": last_hidden_state,
            "pos_logits": pos_logits,
            "hidden_states": hidden_states
        }

# Initialize with your model and specified POS vocabulary size
pos_vocab_size = 50  # Adjust based on actual POS vocabulary size
multi_task_model = MultiTaskBERTWithPOS(base_model=base_model, pos_vocab_size=pos_vocab_size).to(device)

In [40]:
class ThaiMultiTaskDataset(Dataset):
    def __init__(self, texts, pos_tags, labels):
        self.texts = texts
        self.pos_tags = pos_tags
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens, pos_tags = tokenize_and_pos_tag(self.texts[idx])
        pos_indices = pos_to_indices(pos_tags)

        encoding = tokenizer(tokens, is_split_into_words=True, padding="max_length", truncation=True, max_length=128)
        input_ids = torch.tensor(encoding['input_ids'])
        attention_mask = torch.tensor(encoding['attention_mask'])
        pos_labels = torch.tensor(pos_indices[:128] + [0] * (128 - len(pos_indices)))  # Padding to max length
        label = torch.tensor(self.labels[idx])  # Ensure labels are included

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'pos_labels': pos_labels,
            'labels': label  # Add labels for hate speech classification
        }

# Example usage
texts = df['texts'].tolist()
pos_tags = [pos_to_indices(pos_tag(word_tokenize(text))) for text in texts]
labels = df['weak_label'].tolist()  # Ensure this is the hate speech label column

thai_dataset = ThaiMultiTaskDataset(texts, pos_tags, labels)
thai_dataloader = DataLoader(thai_dataset, batch_size=16, shuffle=True)


In [41]:
import torch
from torch.utils.data import DataLoader
import numpy as np

# Define a function to extract embeddings for a dataset
def extract_embeddings(model, dataloader):
    model.eval()
    embeddings, labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label = batch['labels'].to(device)

            # Get the outputs by calling the model directly
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Adjust this to access the last hidden state appropriately
            last_hidden_state = outputs["last_hidden_state"][:, 0, :]  # CLS token embedding
            embeddings.append(last_hidden_state)
            labels.append(label)

    embeddings = torch.cat(embeddings)
    labels = torch.cat(labels)
    return embeddings, labels


# Extract embeddings from the Thai dataset
thai_embeddings, thai_labels = extract_embeddings(multi_task_model, thai_dataloader)

# Calculate prototypes for each class (hate and non-hate)
def calculate_prototypes(embeddings, labels, num_classes=2):
    prototypes = []
    for label in range(num_classes):
        class_embeddings = embeddings[labels == label]
        prototype = class_embeddings.mean(dim=0)
        prototypes.append(prototype)
    return torch.stack(prototypes)

thai_prototypes = calculate_prototypes(thai_embeddings, thai_labels)
print("Thai prototypes:", thai_prototypes)


Thai prototypes: tensor([[-0.5080,  0.1452, -0.0776,  ..., -0.0991, -0.9618,  0.0271],
        [ 0.1716, -0.0416,  0.7509,  ...,  0.2973,  0.7014, -0.3484]],
       device='cuda:0')


In [42]:
import torch
from torch.optim import RMSprop
from torch.nn import functional as F

# Updated hyperparameters
meta_learning_rate = 1e-4
support_set_size = 64
query_set_size = 32
num_episodes = 200
temperature = 0.3  # Reduced temperature for sharper separation
margin = 0.5  # Reduced margin in contrastive loss

# Initialize optimizer
meta_optimizer = RMSprop(multi_task_model.parameters(), lr=meta_learning_rate)

# Function to compute prototypes
def compute_prototypes(embeddings, labels):
    unique_labels = torch.unique(labels)
    prototypes = []
    for label in unique_labels:
        class_embeddings = embeddings[labels == label]
        class_prototype = class_embeddings.mean(dim=0)
        prototypes.append(class_prototype)
    return torch.stack(prototypes)

# Function for contrastive loss
def contrastive_loss(embeddings, prototypes, labels, margin=margin):
    positive_distances = torch.norm(embeddings - prototypes[labels], dim=1)
    negative_distances = torch.min(torch.norm(embeddings.unsqueeze(1) - prototypes, dim=2), dim=1)[0]
    return F.relu(margin + positive_distances - negative_distances).mean()

# Token dropout function
def add_token_dropout(embeddings, dropout_rate=0.1):
    mask = torch.bernoulli(torch.full(embeddings.shape, 1 - dropout_rate)).to(embeddings.device)
    return embeddings * mask

# Meta-learning loop
for episode in range(num_episodes):
    support_indices = np.random.choice(len(thai_labels), size=support_set_size, replace=False)
    query_indices = np.random.choice(len(thai_labels), size=query_set_size, replace=False)

    support_embeddings = thai_embeddings[support_indices].clone().detach().requires_grad_(True)
    support_labels = thai_labels[support_indices]
    query_embeddings = thai_embeddings[query_indices].clone().detach().requires_grad_(True)
    query_labels = thai_labels[query_indices]

    # Apply token dropout
    support_embeddings = add_token_dropout(support_embeddings, dropout_rate=0.1)

    # Compute prototypes
    prototypes = compute_prototypes(support_embeddings, support_labels)

    # Distance calculation with temperature scaling
    distances = 1 - F.cosine_similarity(query_embeddings.unsqueeze(1), prototypes.unsqueeze(0), dim=2) / temperature
    logits = -distances

    # Calculate loss with adjusted contrastive loss and gradient penalty
    contrastive = contrastive_loss(query_embeddings, prototypes, query_labels)
    grad = torch.autograd.grad(contrastive, query_embeddings, create_graph=True)[0]
    grad_penalty = (grad.norm(2, dim=1) ** 2).mean() * 0.05  # Reduced penalty weight
    loss = contrastive + grad_penalty

    # Optimization step
    meta_optimizer.zero_grad()
    loss.backward()
    meta_optimizer.step()

    if (episode + 1) % 10 == 0:
        print(f"Episode {episode + 1}/{num_episodes}, Meta-Loss: {loss.item()}")


Episode 10/200, Meta-Loss: 0.5
Episode 20/200, Meta-Loss: 0.5
Episode 30/200, Meta-Loss: 0.5
Episode 40/200, Meta-Loss: 0.5
Episode 50/200, Meta-Loss: 0.5
Episode 60/200, Meta-Loss: 0.5
Episode 70/200, Meta-Loss: 0.5
Episode 80/200, Meta-Loss: 1.1078805923461914
Episode 90/200, Meta-Loss: 0.5
Episode 100/200, Meta-Loss: 0.5
Episode 110/200, Meta-Loss: 1.103718638420105
Episode 120/200, Meta-Loss: 0.5
Episode 130/200, Meta-Loss: 0.5
Episode 140/200, Meta-Loss: 0.5
Episode 150/200, Meta-Loss: 0.5
Episode 160/200, Meta-Loss: 0.5
Episode 170/200, Meta-Loss: 0.5
Episode 180/200, Meta-Loss: 0.5
Episode 190/200, Meta-Loss: 0.5
Episode 200/200, Meta-Loss: 0.5


In [43]:
import os
import torch
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Specify the directory to save the model and create it if it doesn't exist
save_directory = '/content/drive/MyDrive/cross_lingual_meta_learning_model_with_prototypes'
os.makedirs(save_directory, exist_ok=True)

# Save the model state dictionary
torch.save(multi_task_model.state_dict(), f"{save_directory}/cross_lingual_meta_learning_model_state.pth")

# Save additional data (prototypes and other components if needed)
torch.save({
    'prototypes': prototypes,  # Save the last computed prototypes
    'meta_optimizer_state': meta_optimizer.state_dict()  # Save optimizer state if needed for further training
}, f"{save_directory}/cross_lingual_meta_learning_additional_data.pth")

print(f"Model and related data saved to {save_directory}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model and related data saved to /content/drive/MyDrive/cross_lingual_meta_learning_model_with_prototypes


In [44]:
import pandas as pd

# Load the CSV containing the hate keywords for each language
keywords_path = '/content/Toxic_Words_translations.csv'  # Replace with your actual file path
keywords_df = pd.read_csv(keywords_path)

# Extract keywords for each language as lists, removing any NaN values
keywords_lao = keywords_df['Lao'].dropna().tolist()
keywords_khmer = keywords_df['Khmer'].dropna().tolist()
keywords_shan = keywords_df['Shan'].dropna().tolist()

# Verify the keywords were loaded correctly
print("Lao Keywords:", keywords_lao)
print("Khmer Keywords:", keywords_khmer)
print("Shan Keywords:", keywords_shan)


Lao Keywords: ['ຄວາຍ', 'ບໍ່ສັດຕະສົງ', 'ຫມາ', 'ຕໍແຫລ', 'ໂງ່', 'ຕ່ໍາ', 'ບ້າ', 'ຫມາຍຕິກ', 'ຊົ່ວ', 'ກະເສີມ', 'ຄວາມຊົ່ວ', 'ປັກຫິນດອນ', 'ໂກງ', 'ສັດ', 'ອອກຄໍາຄິດເຫັນ', 'ລັກຊະນະ', 'ຫອຍfish', 'ລັງແສງ', 'ປວກ', 'ຂີ້ຄ້ານ', 'ຂີ້ເຫຍື້ອ', 'ກັມ', 'ເສັ້ນປະສາດ', 'ດອກ', 'ສົ້ນ', 'ດິນແຮ່', 'ພົນໄພ່', 'ສິ່ງທີ່ ', 'ຫອກ', 'ສາມາດ', 'ຂ້າ', 'ອັດຕາສ່ວນ', 'ຂີ້ຊ່ານ', 'ນ້ໍາເຂີນ', 'ກະຫລີ່', 'ອອກຄໍາຄິດເຫັນ', 'ເວນແຢກ', 'ໄກ່', 'ຂີ້ປິບ', 'ເສຽກໍາ', 'ສັດທີ່', 'ຊັກຊ້າ', 'ແຮດ/ໝາ', 'ຄວາມຊົ່ວ']
Khmer Keywords: ['រកបី', 'មិនស្មោះត្រង់', 'ឆ្កេ', 'កាមតេ', 'ផ្លីផ្លើ', 'តាប', 'ឈ្កូ', 'ចៃ', 'អារកក់', 'ការីឈី', 'ដេលក្ផានសីលធ៍ម', 'ផាប', 'ថ្ពាល់', 'សតវ', 'រមាំn', 'ចរិតរបា', 'សែលហ្វី', 'តារុងរឿង', 'កន្ដ្យេវ', 'scurlilog', 'សមនល់បិផ្ទហបាយ', 'វាសនា', 'សរសៃរបសាត', 'ផ្កា', 'កេងចើង', 'ការរបយ័បន៍', 'ការធេវីបេកចិត៍', 'អ្វីដែល heck នេះ!', 'លមបេង', 'អាច', 'សម្លាប់', 'សមាមារត', 'របសផាប', 'ញិរបុត', 'ស្លត់', 'រមាំn', 'កសំណឹក', 'សាកសម', 'កមនិតឥតខ្លឹមសារ', 'ផាបល្អ', 'ចាហតុរ', 'បានថយចុះ', 'សតវរមាស', 'ដេលក្ផានសីលធ៍ម']
Shan Keywords: ['ကြှဲ', 'မရိုးမသားဖ

In [45]:
import torch
from torch.utils.data import Dataset, DataLoader
import random

# Generating synthetic few-shot datasets using keywords
def generate_few_shot_dataset(keywords, num_samples=10, non_hate_text="this is a neutral sentence"):
    """Generate a few-shot dataset using keywords for hate and a generic non-hate sentence."""
    dataset = []
    # Positive samples using keywords
    for _ in range(num_samples // 2):
        text = random.choice(keywords)
        dataset.append((text, 1))  # Label 1 for hate
    # Negative samples
    for _ in range(num_samples // 2):
        dataset.append((non_hate_text, 0))  # Label 0 for non-hate
    return dataset

# Generating few-shot datasets for each language
few_shot_data_lao = generate_few_shot_dataset(keywords_lao)
few_shot_data_khmer = generate_few_shot_dataset(keywords_khmer)
few_shot_data_shan = generate_few_shot_dataset(keywords_shan)

# Wrap into DataLoader for batch processing
class FewShotDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, label = self.data[idx]
        encoding = self.tokenizer(text, return_tensors="pt", padding="max_length", max_length=128, truncation=True)
        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()
        return {"input_ids": input_ids, "attention_mask": attention_mask, "label": torch.tensor(label)}


# Load the tokenizer (use the name of your pre-trained model)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW")  # Replace with your specific model if different
batch_size = 4

few_shot_loader_lao = DataLoader(FewShotDataset(few_shot_data_lao, tokenizer), batch_size=batch_size)
few_shot_loader_khmer = DataLoader(FewShotDataset(few_shot_data_khmer, tokenizer), batch_size=batch_size)
few_shot_loader_shan = DataLoader(FewShotDataset(few_shot_data_shan, tokenizer), batch_size=batch_size)


In [46]:
def update_prototypes_with_few_shot(model, dataloader, initial_prototypes, embedding_dim=768):
    """Refine prototypes using few-shot data."""
    model.eval()
    refined_prototypes = initial_prototypes.clone()

    num_classes = initial_prototypes.shape[0]
    refined_prototypes = refined_prototypes.to(device)
    all_embeddings, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Attempt to use pooled output if available, otherwise use [CLS] token embedding
            if 'pooler_output' in outputs and outputs['pooler_output'] is not None:
                embeddings = outputs['pooler_output']
            elif 'last_hidden_state' in outputs:
                # Use [CLS] token (first token) from last hidden state as a fallback
                embeddings = outputs['last_hidden_state'][:, 0, :]  # Shape: (batch_size, embedding_dim)
            else:
                raise ValueError("Neither 'pooler_output' nor 'last_hidden_state' found in model outputs")

            if embeddings.shape[1] != embedding_dim:
                raise ValueError(f"Embedding dimension mismatch. Expected {embedding_dim}, got {embeddings.shape[1]}")

            all_embeddings.append(embeddings)
            all_labels.append(labels)

    # Concatenate all embeddings and labels
    all_embeddings = torch.cat(all_embeddings)
    all_labels = torch.cat(all_labels)

    # Update prototypes based on embeddings
    for label in range(num_classes):
        class_embeddings = all_embeddings[all_labels == label]
        if class_embeddings.shape[0] > 0:  # Check that there are embeddings for this class
            refined_prototypes[label] = class_embeddings.mean(dim=0)

    return refined_prototypes


In [47]:
# Update prototypes for each language
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_dim = 768  # Expected embedding dimension

# Ensure prototypes tensor matches the expected size (e.g., 2 classes for hate/non-hate, 768 embedding size)
prototypes = torch.zeros((2, embedding_dim), device=device)

updated_prototypes_lao = update_prototypes_with_few_shot(multi_task_model, few_shot_loader_lao, prototypes, embedding_dim)
updated_prototypes_khmer = update_prototypes_with_few_shot(multi_task_model, few_shot_loader_khmer, prototypes, embedding_dim)
updated_prototypes_shan = update_prototypes_with_few_shot(multi_task_model, few_shot_loader_shan, prototypes, embedding_dim)


In [48]:
# Save updated prototypes
torch.save({
    'prototypes_lao': updated_prototypes_lao,
    'prototypes_khmer': updated_prototypes_khmer,
    'prototypes_shan': updated_prototypes_shan
}, "/content/drive/MyDrive/cross_lingual_meta_learning_model_with_prototypes/updated_prototypes_few_shot.pth")

print("Few-shot fine-tuning complete and refined prototypes saved.")


Few-shot fine-tuning complete and refined prototypes saved.


In [49]:
import torch
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Load the saved model state dictionary
model_path = '/content/drive/MyDrive/cross_lingual_meta_learning_model_with_prototypes/cross_lingual_meta_learning_model_state.pth'
multi_task_model.load_state_dict(torch.load(model_path, map_location=device))
multi_task_model.to(device)
multi_task_model.eval()

# Load updated prototypes
prototype_path = '/content/drive/MyDrive/cross_lingual_meta_learning_model_with_prototypes/updated_prototypes_few_shot.pth'
updated_prototypes = torch.load(prototype_path)


<ipython-input-49-4aa89a1aaf21>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  multi_task_model.load_state_dict(torch.load(model_path, map_location=device))
<ipython-input

In [50]:
import pandas as pd
import re

# Define maximum character length
max_length = 200

# Load the datasets
thai_df = pd.read_csv('/content/thai_test_data.csv')
lao_df = pd.read_csv('/content/lao_test_data.csv')
shan_df = pd.read_csv('/content/shan_test_data.csv')

# Function to remove URLs and special characters, and clean text minimally
def basic_clean_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

# Function to truncate text to max length
def truncate_text(text, max_length):
    return text[:max_length] if len(text) > max_length else text

# Apply minimal cleaning and truncating to each dataset without dropping NaN labels
def preprocess_dataframe_basic(df):
    print("Initial Data Sample:\n", df.head())  # Initial data sample
    df['text'] = df['text'].astype(str).apply(basic_clean_text)
    print("Post-Cleaning Data Sample:\n", df.head())  # After cleaning
    df['text'] = df['text'].apply(lambda x: truncate_text(x, max_length))
    print("Post-Truncating Data Sample:\n", df.head())  # After truncating
    return df

# Preprocess each dataframe without filtering out NaN labels
thai_df_cleaned = preprocess_dataframe_basic(thai_df)
lao_df_cleaned = preprocess_dataframe_basic(lao_df)
shan_df_cleaned = preprocess_dataframe_basic(shan_df)

# Final samples for verification
print("Final Thai Data Sample:\n", thai_df_cleaned.head())
print("\nFinal Lao Data Sample:\n", lao_df_cleaned.head())
print("\nFinal Shan Data Sample:\n", shan_df_cleaned.head())


Initial Data Sample:
                                                 text  label
0  ไป จอง มา แล้ วนา จา   mitsubishi   attrage   ...    NaN
1  เปิด ศักราช ใหม่ !   นายกฯ   แถลงข่าว ก่อน การ...    NaN
2                      บัตรสมาชิก ลด ได้ อีก ไหม คับ    NaN
3                          สนใจ   new   mazda 2 ครับ    NaN
4                                                 😍😍    NaN
Post-Cleaning Data Sample:
                                                 text  label
0  ไป จอง มา แล้ วนา จา mitsubishi attrage ได้ หล...    NaN
1  เปิด ศักราช ใหม่ ! นายกฯ แถลงข่าว ก่อน การแข่ง...    NaN
2                      บัตรสมาชิก ลด ได้ อีก ไหม คับ    NaN
3                              สนใจ new mazda 2 ครับ    NaN
4                                                 😍😍    NaN
Post-Truncating Data Sample:
                                                 text  label
0  ไป จอง มา แล้ วนา จา mitsubishi attrage ได้ หล...    NaN
1  เปิด ศักราช ใหม่ ! นายกฯ แถลงข่าว ก่อน การแข่ง...    NaN
2                   

In [82]:
import pandas as pd
import re

# Load the toxic keywords from the CSV file
keywords_df = pd.read_csv('/content/Toxic_Words_translations.csv')

# Extract keywords for each language
def extract_keywords_for_language(df, language_column):
    # Drop NaN values and convert to a list
    keywords = df[language_column].dropna().tolist()
    # Ensure all keywords are strings
    keywords = [str(keyword) for keyword in keywords]
    return keywords

# Extract keywords for each language
thai_toxic_keywords = extract_keywords_for_language(keywords_df, 'Thai')
lao_toxic_keywords = extract_keywords_for_language(keywords_df, 'Lao')
shan_toxic_keywords = extract_keywords_for_language(keywords_df, 'Shan')

# Function to label data based on toxic keywords
def label_data_using_keywords(df, toxic_keywords):
    # Compile regex pattern for faster matching
    pattern = re.compile(r'\b(' + '|'.join(map(re.escape, toxic_keywords)) + r')\b', re.IGNORECASE)

    # Function to check if text contains any toxic keywords
    def check_toxic(text):
        if pattern.search(text):
            return 1  # Hate
        else:
            return 0  # Non-hate

    # Apply the function to the 'text' column
    df['label'] = df['text'].apply(check_toxic)
    return df

# Apply labeling to each dataset
thai_df_labeled = label_data_using_keywords(thai_df_cleaned, thai_toxic_keywords)
lao_df_labeled = label_data_using_keywords(lao_df_cleaned, lao_toxic_keywords)
shan_df_labeled = label_data_using_keywords(shan_df_cleaned, shan_toxic_keywords)

# Verify the new label distribution
print("Thai Data Label Distribution:\n", thai_df_labeled['label'].value_counts())
print("\nLao Data Label Distribution:\n", lao_df_labeled['label'].value_counts())
print("\nShan Data Label Distribution:\n", shan_df_labeled['label'].value_counts())


Thai Data Label Distribution:
 label
0    23623
1    17943
Name: count, dtype: int64

Lao Data Label Distribution:
 label
0    80
1     1
Name: count, dtype: int64

Shan Data Label Distribution:
 label
0    74
Name: count, dtype: int64


In [84]:
import pandas as pd

# Function to balance the dataset using oversampling
def oversample_dataset(df):
    # Separate minority and majority classes
    hate_df = df[df['label'] == 1]
    non_hate_df = df[df['label'] == 0]

    # Calculate the number of samples in each class
    num_hate = len(hate_df)
    num_non_hate = len(non_hate_df)

    # Check if either class is empty
    if num_hate == 0 or num_non_hate == 0:
        print("Warning: One of the classes is empty. Returning the original dataset.")
        return df.copy()

    # Determine the class with the maximum samples
    max_samples = max(num_hate, num_non_hate)

    # Oversample the minority class
    if num_hate < num_non_hate:
        hate_df_oversampled = hate_df.sample(n=max_samples, replace=True, random_state=42)
        non_hate_df_oversampled = non_hate_df
    elif num_non_hate < num_hate:
        non_hate_df_oversampled = non_hate_df.sample(n=max_samples, replace=True, random_state=42)
        hate_df_oversampled = hate_df
    else:
        # Classes are already balanced
        oversampled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
        return oversampled_df

    # Combine and shuffle the dataset
    oversampled_df = pd.concat([hate_df_oversampled, non_hate_df_oversampled]).sample(
        frac=1, random_state=42).reset_index(drop=True)

    return oversampled_df

# Apply the oversampling function to your datasets
thai_df_oversampled = oversample_dataset(thai_df_labeled)
lao_df_oversampled = oversample_dataset(lao_df_labeled)
shan_df_oversampled = oversample_dataset(shan_df_labeled)

# Verify the new label distribution
print("Thai Data Label Distribution:\n", thai_df_oversampled['label'].value_counts())
print("\nLao Data Label Distribution:\n", lao_df_oversampled['label'].value_counts())
print("\nShan Data Label Distribution:\n", shan_df_oversampled['label'].value_counts())


Thai Data Label Distribution:
 label
0    23623
1    23623
Name: count, dtype: int64

Lao Data Label Distribution:
 label
0    80
1    80
Name: count, dtype: int64

Shan Data Label Distribution:
 label
0    74
Name: count, dtype: int64


In [85]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer

# Initialize tokenizer (ensure it matches the model you fine-tuned)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW")

class LanguageTestDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=200):
        self.texts = texts
        self.labels = labels.fillna(0).astype(int)  # Convert NaN to 0 for non-hate
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Indented block for __getitem__ method
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long)  # Changed 'label' to 'labels'
        }




# Create dataset and dataloader for each language
def create_dataloader(df, tokenizer, batch_size=8):
    dataset = LanguageTestDataset(df['text'], df['label'], tokenizer)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

thai_dataloader = create_dataloader(thai_df_cleaned, tokenizer)
lao_dataloader = create_dataloader(lao_df_cleaned, tokenizer)
shan_dataloader = create_dataloader(shan_df_cleaned, tokenizer)


print("DataLoaders created for Thai, Lao, and Shan datasets.")

DataLoaders created for Thai, Lao, and Shan datasets.


In [86]:
import torch
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Load the saved model state dictionary
model_path = '/content/drive/MyDrive/cross_lingual_meta_learning_model_with_prototypes/cross_lingual_meta_learning_model_state.pth'
multi_task_model.load_state_dict(torch.load(model_path, map_location=device))
multi_task_model.to(device)
multi_task_model.eval()

# Load updated prototypes
prototype_path = '/content/drive/MyDrive/cross_lingual_meta_learning_model_with_prototypes/updated_prototypes_few_shot.pth'
updated_prototypes = torch.load(prototype_path)


AttributeError: 'str' object has no attribute 'load_state_dict'

In [70]:
def calculate_prototypes_with_weights(embeddings, labels, num_classes=2):
    """Calculate weighted prototypes for each class to emphasize core samples."""
    prototypes = []
    for label in range(num_classes):
        class_embeddings = embeddings[labels == label]
        if class_embeddings.shape[0] > 0:
            # Center the embeddings and compute distances
            center = class_embeddings.mean(dim=0, keepdim=True)
            distances = torch.norm(class_embeddings - center, dim=1)

            # Calculate weights: closer embeddings have higher weights
            weights = 1 / (distances + 1e-6)
            weighted_embeddings = class_embeddings * weights.unsqueeze(1)
            weighted_prototype = weighted_embeddings.sum(dim=0) / weights.sum()
            prototypes.append(weighted_prototype)
        else:
            prototypes.append(torch.zeros(embeddings.shape[1]))  # Handle empty classes

    return torch.stack(prototypes)




In [71]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch

def evaluate_with_cosine_similarity(model, dataloader, prototypes):
    """Evaluate using cosine similarity to prototypes."""
    model.eval()
    all_preds, all_labels = [], []

    prototypes = prototypes.to(device)  # Ensure prototypes are on the correct device

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label = batch['labels'].to(device)  # Changed 'label' to 'labels'

            # Get embeddings from the model
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Handle different output structures
            if hasattr(outputs, 'last_hidden_state'):
                last_hidden_state = outputs.last_hidden_state
            elif isinstance(outputs, tuple):
                last_hidden_state = outputs[0]  # For models that return tuples
            else:
                raise ValueError("Unexpected model output structure.")

            embeddings = last_hidden_state[:, 0, :]  # Use [CLS] token representation

            # Calculate cosine similarity to prototypes
            similarities = torch.nn.functional.cosine_similarity(
                embeddings.unsqueeze(1), prototypes.unsqueeze(0), dim=2
            )
            preds = torch.argmax(similarities, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(label.cpu().numpy())

    # Calculate accuracy and F1-score
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average="macro")
    report = classification_report(all_labels, all_preds, target_names=["Non-hate", "Hate"])

    return accuracy, macro_f1, report


In [72]:
def extract_embeddings(model, dataloader):
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            batch_labels = batch['labels'].to(device)  # Changed 'label' to 'labels'

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
            # Assuming you use the last hidden state as embeddings
            hidden_states = outputs.hidden_states[-1][:, 0, :]  # Use the [CLS] token representation
            embeddings.append(hidden_states.cpu())
            labels.append(batch_labels.cpu())
    embeddings = torch.cat(embeddings)
    labels = torch.cat(labels)
    return embeddings, labels


In [78]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize tokenizer (ensure it matches the model you fine-tuned)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Cross_Lingual_Meta_Learning_NEW")

# Define the dataset class
class LanguageTestDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=200):
        self.texts = texts.reset_index(drop=True)
        self.labels = labels.fillna(0).astype(int).reset_index(drop=True)  # Convert NaN to 0 for non-hate
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long)  # Use 'labels' as the key
        }

# Create dataset and dataloader for each language
def create_dataloader(df, tokenizer, batch_size=8):
    dataset = LanguageTestDataset(df['text'], df['label'], tokenizer)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Replace these with your actual cleaned DataFrames
# thai_df_cleaned = ...
# lao_df_cleaned = ...
# shan_df_cleaned = ...

thai_dataloader = create_dataloader(thai_df_cleaned, tokenizer)
lao_dataloader = create_dataloader(lao_df_cleaned, tokenizer)
shan_dataloader = create_dataloader(shan_df_cleaned, tokenizer)

print("DataLoaders created for Thai, Lao, and Shan datasets.")

# Load your trained model
# Ensure that the model is configured to output hidden states
multi_task_model = AutoModel.from_pretrained("/content/drive/MyDrive/Cross_Lingual_Meta_Learning", output_hidden_states=True)
multi_task_model.to(device)

# Define the embedding extraction function
def extract_embeddings(model, dataloader):
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            batch_labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
            # Access the last hidden state
            hidden_states = outputs.hidden_states[-1][:, 0, :]  # Use the [CLS] token representation
            embeddings.append(hidden_states.cpu())
            labels.append(batch_labels.cpu())
    embeddings = torch.cat(embeddings)
    labels = torch.cat(labels)
    return embeddings, labels

# Define the function to calculate weighted prototypes
def calculate_prototypes_with_weights(embeddings, labels, num_classes=2):
    """Calculate weighted prototypes for each class to emphasize core samples."""
    prototypes = []
    for label in range(num_classes):
        class_embeddings = embeddings[labels == label]
        if class_embeddings.shape[0] > 0:
            # Center the embeddings and compute distances
            center = class_embeddings.mean(dim=0, keepdim=True)
            distances = torch.norm(class_embeddings - center, dim=1)

            # Calculate weights: closer embeddings have higher weights
            weights = 1 / (distances + 1e-6)
            weighted_embeddings = class_embeddings * weights.unsqueeze(1)
            weighted_prototype = weighted_embeddings.sum(dim=0) / weights.sum()
            prototypes.append(weighted_prototype)
        else:
            prototypes.append(torch.zeros(embeddings.shape[1]))  # Handle empty classes

    return torch.stack(prototypes)

# Define the evaluation function
def evaluate_with_cosine_similarity(model, dataloader, prototypes):
    """Evaluate using cosine similarity to prototypes."""
    model.eval()
    all_preds, all_labels = [], []

    prototypes = prototypes.to(device)  # Ensure prototypes are on the correct device

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            batch_labels = batch['labels'].to(device)

            # Get embeddings from the model
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)

            # Access hidden states
            if hasattr(outputs, 'hidden_states'):
                hidden_states = outputs.hidden_states[-1]
            elif isinstance(outputs, tuple):
                hidden_states = outputs[-1]  # For models that return tuples
            else:
                raise ValueError("Unexpected model output structure.")

            embeddings = hidden_states[:, 0, :]  # Use [CLS] token representation

            # Calculate cosine similarity to prototypes
            similarities = torch.nn.functional.cosine_similarity(
                embeddings.unsqueeze(1), prototypes.unsqueeze(0), dim=2
            )
            preds = torch.argmax(similarities, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())

    # Calculate accuracy and F1-score
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average="macro")
    report = classification_report(all_labels, all_preds, target_names=["Non-hate", "Hate"])

    return accuracy, macro_f1, report

# Step 1: Extract embeddings for each language
print("Extracting embeddings for Thai dataset...")
thai_embeddings, thai_labels = extract_embeddings(multi_task_model, thai_dataloader)
print("Extracting embeddings for Lao dataset...")
lao_embeddings, lao_labels = extract_embeddings(multi_task_model, lao_dataloader)
print("Extracting embeddings for Shan dataset...")
shan_embeddings, shan_labels = extract_embeddings(multi_task_model, shan_dataloader)

# Step 2: Generate prototypes using the new weighted approach
print("Calculating weighted prototypes for Thai dataset...")
updated_prototypes_thai = calculate_prototypes_with_weights(thai_embeddings, thai_labels, num_classes=2)
print("Calculating weighted prototypes for Lao dataset...")
updated_prototypes_lao = calculate_prototypes_with_weights(lao_embeddings, lao_labels, num_classes=2)
print("Calculating weighted prototypes for Shan dataset...")
updated_prototypes_shan = calculate_prototypes_with_weights(shan_embeddings, shan_labels, num_classes=2)

# Step 3: Use the refined prototypes in evaluation
print("\nEvaluating Lao Test Set...")
accuracy, macro_f1, report = evaluate_with_cosine_similarity(
    multi_task_model, lao_dataloader, updated_prototypes_lao
)
print(f"Accuracy: {accuracy}\nMacro F1: {macro_f1}\n{report}")

print("\nEvaluating Shan Test Set...")
accuracy, macro_f1, report = evaluate_with_cosine_similarity(
    multi_task_model, shan_dataloader, updated_prototypes_shan
)
print(f"Accuracy: {accuracy}\nMacro F1: {macro_f1}\n{report}")


DataLoaders created for Thai, Lao, and Shan datasets.
Extracting embeddings for Thai dataset...
Extracting embeddings for Lao dataset...
Extracting embeddings for Shan dataset...
Calculating weighted prototypes for Thai dataset...
Calculating weighted prototypes for Lao dataset...
Calculating weighted prototypes for Shan dataset...

Evaluating Lao Test Set...
Accuracy: 0.7037037037037037
Macro F1: 0.5533088235294118
              precision    recall  f1-score   support

    Non-hate       0.95      0.71      0.81        73
        Hate       0.19      0.62      0.29         8

    accuracy                           0.70        81
   macro avg       0.57      0.67      0.55        81
weighted avg       0.87      0.70      0.76        81


Evaluating Shan Test Set...
Accuracy: 0.5135135135135135
Macro F1: 0.4322250639386189
              precision    recall  f1-score   support

    Non-hate       0.94      0.49      0.65        67
        Hate       0.13      0.71      0.22         7

  